In [57]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

In [19]:
violations = pd.read_csv('data/violations.csv')
trainlabels = pd.read_csv('data/train_labels.csv')

/home/ishaan/ishaan/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
violations.head()

,TicketID,TicketNumber,AgencyName,ViolName,ViolationStreetNumber,ViolationStreetName,MailingStreetNumber,MailingStreetName,MailingCity,MailingState,...,CleanUpCost,JudgmentAmt,PaymentStatus,Void,ViolationCategory,ViolationAddress,MailingAddress,V_LAT,V_LONG,V_addr
0,26288,05000001DAH,Department of Public Works,"Group, LLC, Grand Holding",2566,GRAND BLVD,743,"Beaubien, Ste. 201",Detroit,MI,...,$0.00,$1680.00,PAID IN FULL,0.0,0,"2566 GRAND BLVD\nDetroit, MI\n(42.363182370000...","743 Beaubien\nDetroit, MI 48226\n(42.333730630...",42.363182,-83.091677,"2566 GRAND BLVD,Detroit, MI"
1,19800,05000025DAH,Department of Public Works,"JACKSON, RAECHELLE",19014,ASHTON,20501,HEYDEN,DETROIT,MI,...,$0.00,$140.00,NO PAYMENT APPLIED,0.0,0,"19014 ASHTON\nDetroit, MI\n(42.429390762000025...","20501 HEYDEN\nDETROIT, MI 48219\n(42.442177633...",42.429391,-83.220394,"19014 ASHTON,Detroit, MI"
2,19804,05000026DAH,Department of Public Works,"TALTON, CAROL ANN",18735,STAHELIN,18735,STAHELI N,DETROIT,MI,...,$0.00,$140.00,PAID IN FULL,0.0,0,"18735 STAHELIN\nDetroit, MI\n(42.4287074590000...","18735 STAHELI N\nDETROIT, MI 48219\n(42.428707...",42.428707,-83.227548,"18735 STAHELIN,Detroit, MI"
3,20208,05000027DAH,Department of Public Works,"BONNER, DARRYL E.",20125,MONICA,25335,PEEKSKILL,SOUTHFIELD,MI,...,$0.00,$140.00,NO PAYMENT APPLIED,0.0,0,"20125 MONICA\nDetroit, MI\n(42.44169828400004,...","25335 PEEKSKILL\nSOUTHFIELD, MI 48043\n(42.475...",42.441698,-83.145018,"20125 MONICA,Detroit, MI"
4,20211,05000028DAH,Department of Public Works,"GREGORY, JAMES LEE",17397,PRAIRIE,17397,PRAIRIE,DETROIT,MI,...,$0.00,$140.00,PAID IN FULL,0.0,0,"17397 PRAIRIE\nDetroit, MI\n(42.42031769500005...","17397 PRAIRIE\nDETROIT, MI 48221\n(42.42031769...",42.420318,-83.145328,"17397 PRAIRIE,Detroit, MI"


In [21]:
trainlabels.head()

,building_id,LAT,LONG,label
0,4,42.433953,-83.098835,blighted
1,6,42.335282,-83.072696,blighted
2,9,42.389837,-83.019746,blighted
3,11,42.329588,-83.156621,blighted
4,14,42.411072,-82.913786,blighted


In [41]:
trainlabels.loc[2,'building_id']

9

In [42]:
def distance(pt,c):
    pt = pt*np.pi/180
    c = c*np.pi/180
    lat,lng = c.T[0],c.T[1]
    deltay = lat - pt[0]
    deltax = (lng - pt[1])*np.cos((lat+pt[0])/2)
    R = 6371e3
    d = R*np.sqrt(np.power(deltax,2)+np.power(deltay,2))
    return d.min(), np.argmin(d)

def findbuilding(row,buildings):
    lat,long = row['V_LAT'],row['V_LONG']
    dis, b_id = distance(np.array([lat,long]),np.array(buildings[['LAT','LONG']]))
    if(dis<=100):
        return buildings.loc[b_id,'building_id']
    else:
        return np.nan

In [43]:
violations['building_id'] = violations.apply(lambda row: findbuilding(row,trainlabels),axis=1)

In [44]:
violationcounts =  violations.dropna(axis=0,subset=['building_id']).groupby('building_id').size()

In [45]:
violationcounts = pd.DataFrame(violationcounts)

In [46]:
violationcounts = violationcounts.rename(columns = {0:"count"})

In [47]:
violationcounts.reset_index(inplace=True)

In [48]:
violationcounts = violationcounts.astype(int)

In [52]:
violationcounts.head()

,building_id,count
0,2,1
1,4,66
2,6,18
3,9,17
4,11,89


In [55]:
violationcounts = violationcounts.merge(trainlabels,on='building_id')

In [56]:
violationcounts.head()

,building_id,count,LAT,LONG,label
0,2,1,42.442180,-83.285080,not_blighted
1,4,66,42.433953,-83.098835,blighted
2,6,18,42.335282,-83.072696,blighted
3,9,17,42.389837,-83.019746,blighted
4,11,89,42.329588,-83.156621,blighted


In [67]:
X = np.array(violationcounts['count']).reshape(-1,1)
model = LogisticRegression(solver='lbfgs')
results = cross_validate(model,X,violationcounts['label'],cv=5)

In [68]:
results

{'fit_time': array([0.23099232, 0.01994944, 0.01839137, 0.01759338, 0.01752949]),
 'score_time': array([0.0041101 , 0.00235724, 0.00239038, 0.00238299, 0.00233531]),
 'test_score': array([0.59790941, 0.54425087, 0.56376307, 0.61881533, 0.62787456])}